Imports

In [ ]:
import datetime
import pandas as pd
from pandas import DataFrame
from openpyxl import load_workbook,worksheet,Workbook

Constants

In [2]:
MYDIR= r"C:\Projects\2022\Michaels_Code\grundfos-express-tools\bronze impeller removal\input files"
OUTPUT_DIR=r"C:\Projects\2022\Michaels_Code\grundfos-express-tools\bronze impeller removal\output files"

PNs to be Removed

In [ ]:
list_of_pns = [
    96699290, 97775274, 96699299, 97775277, 96778078,
    97780992, 96699305, 96769184, 97778033, 96769190,
    96769205, 97778039, 96769256, 96896891, 96769259,
    96769271, 97780970, 96769280, 97780973
]

str_list = [str(numstr) for numstr in list_of_pns]

Justification Note

In [ ]:
def add_removal_note():
    timeStamp = datetime.datetime.now().strftime("%m-%d-%Y")
    reason = input("Reason for removal: ")
    authority = input("Who authorized/requested this change? ")
    return timeStamp + " " + reason + " per " + authority


Geeting the First Row of the PSD

In [ ]:
wb=load_workbook(r'C:\Projects\2022\Michaels_Code\grundfos-express-tools\bronze impeller removal\input files\Lbom-ES.xlsx',read_only=True)

In [ ]:
def get_start_row(wb:Workbook,sheet_name:str,min_row:int=None,max_row:int=None,min_col:int=None,max_col:int=None):
    try:
        if not sheet_name in wb.sheetnames:
            raise Exception
        ws=wb[sheet_name]
    except:
        print('Sheet does not exist')
    min_col = min_col or 1
    min_row = min_row or 1
    max_col = max_col or ws.max_column
    max_row = max_row or ws.max_row
    for row in ws.iter_rows(min_row,max_row,min_col,max_col):
        if row[0].value=='[START]':
            return row[0].row-1

psd_startrow=get_start_row(wb,'Impeller',min_row=1,max_col=1)
psd_startcol=0
        
    

Grouping Function

In [ ]:
def group_then_separate_by(psd_data:DataFrame,list_of_cols: list, pn_col: str)->tuple[DataFrame,DataFrame]:
    """list of cols are grouping categories. pn_col is the column that contains pns to find matches on."""
    groups = psd_data.groupby(list_of_cols) # Had to play around with this to get the right groupings

    df_list_to_remove = [] # will hold list of dataframes to be removed. Will concatenate to 1 dataframe later
    df_list_to_keep   = [] # will hold list of dataframes to remain in PSD. Will concatenate to 1 dataframe later

    # Iterates through each sub-group, checking if there is a pn that meets criteria for removal
    for _, frame in groups:
        if any(frame[pn_col].isin(str_list)):   # Finding matching partnumbers to remove
            df_list_to_remove.append(frame)     # Need to add this sub-group to a "removed dataframe"
        else:
            df_list_to_keep.append(frame)       # Need to retain this sub-group, add to a "keep dataframe"

    # Concatenating list of dfs to single dfs.
    removals = pd.concat(df_list_to_remove)
    keep = pd.concat(df_list_to_keep)
        
    return removals, keep

Iterate Through Directory

In [ ]:
import os
import numpy as np
from typing import Callable,Any,ParamSpec,TypeVar,Concatenate
from utils.file_ops import read_files_in_dir


P = ParamSpec('P')

######## START HERE #####
#Workbook contains Sheets and each file contains only one workbook therefore each file contains multiple sheets
#We want to make a list for any given file of the sheets we want to process 
#We can take a single sheet and process over it with the below function



files=read_files_in_dir(MYDIR)
sheetname='Impeller'


for file in files:
    removal_note=add_removal_note()
    data,fname=file
    psd_data=pd.read_excel(data,sheet_name=sheetname,header=psd_startrow-1,dtype={'BOM':str})
    original_psd_bottom_row = len(psd_data) + psd_startrow
    
    print("Opening file for updates: {}".format(os.path.basename(fname)))
    end_row = psd_data[psd_data['Full Data'] == '[END]'].index.to_list()[0]
    psd_data = psd_data.iloc[:end_row]
    group_by_columns = ["Model", "Price ID"]
    removals, keep = group_then_separate_by(psd_data,group_by_columns, "BOM")
    removals.loc[removals["Full Data"] == "[START]", "Full Data"] = ""
    new_row = pd.DataFrame({'ID': removal_note}, index =[0])
    removals = pd.concat([new_row, removals[:]]).reset_index()
    column_list = keep.columns
    removals = removals[column_list]
    removals.sort_values(by=['ID'], inplace=True)
    keep.loc[0,'Full Data'] = np.nan
    keep.sort_values(by=['ID'], inplace=True)
    keep.reset_index(drop=True, inplace=True)
    keep.loc[0,'Full Data'] = "[START]"
    num_rows = len(keep)                      
    keep.loc[num_rows,'Full Data']="[END]"
    append_location = original_psd_bottom_row - len(removals) 
    after_end_row = num_rows + psd_startrow + 2


#Assuming only one sheet for right now
# def before_change_data(dir,sheet_name:list[list[str]],is_same_note=True):
#     files=read_files_in_dir(dir)
#     for i,file in files:
#         sheet=[]
#         note=add_removal_note()
#         for sheet in sheet_name[i]:
#             sheet.append({'header':get_start_row(load_workbook(file[0],read_only=True),sheet_name=sheet,min_row=1,max_col=1)})
#             if not is_same_note:
#                 note=add_removal_note()
            
# before_change_data()


# a=list(before_change_data(MYDIR))


# def change_data(file:tuple[bytes,str,str],psd_data_func:Callable[Concatenate[P],DataFrame],psd_data_args:tuple[list,dict]):
#     removal_note=add_removal_note()
#     data,fname=file
#     psd_data=psd_data_func(data,*psd_data_args[0]) #will have to be called as a list or all have to be the same
#     original_psd_bottom_row = len(psd_data) + psd_startrow
#     print("Opening file for updates: {}".format(os.path.basename(fname)))
#     #...This is where the real processing begins...
#     end_row = psd_data[psd_data['Full Data'] == '[END]'].index.to_list()[0]
#     psd_data = psd_data.iloc[:end_row]
#     group_by_columns = ["Model", "Price ID"]
#     removals, keep = group_then_separate_by(psd_data,group_by_columns, "BOM")
#     removals.loc[removals["Full Data"] == "[START]", "Full Data"] = np.nan
#     new_row = pd.DataFrame({'ID': removal_note}, index =[0])
#     removals = pd.concat([new_row, removals[:]]).reset_index()
#     column_list = keep.columns
#     removals = removals[column_list]
#     removals.sort_values(by=['ID'], inplace=True)
#     keep.loc[0,'Full Data'] = np.nan
#     keep.sort_values(by=['ID'], inplace=True)
#     keep.reset_index(drop=True, inplace=True)
#     keep.loc[0,'Full Data'] = "[START]"
#     return keep,removals,fname



Checking for Misses

In [ ]:
if (len(keep[keep['BOM'].isin(str_list)])) == 0:
    print("Didn't miss any partnumbers. Good to go")
else:
    print("For some reason, the following entries were missed")
    print(keep[keep['BOM'].isin(str_list)])

Return a formatted file

In [ ]:
from openpyxl.styles import PatternFill

outputPath=r'C:\Projects\2022\Michaels_Code\grundfos-express-tools\bronze impeller removal\output files\Revised Lbom-ES.xlsx'

# wb = load_workbook(outputPath)
# ws = wb['Impeller']
# tabName = sheetname + " No Bronze"
# wb.copy_worksheet(ws).title = tabName
# ws_modified = wb[tabName]
# ws_modified.delete_rows(after_end_row, len(removals)-1)
# end_cell=ws.cell(row=after_end_row-1,column=1)
# end_cell.fill=PatternFill(fill_type='solid',start_color="FFCC99",end_color="FFCC99")
# for rows in ws_modified.iter_rows(min_row=append_location+2, max_row=append_location+2, min_col=1, max_col=40):
#     for cell in rows:
#         cell.fill = PatternFill(start_color="FF0000", end_color="FF0000", fill_type="solid")
# wb.save(outputPath)

In [ ]:
# outputPath=r'C:\Projects\2022\Michaels_Code\grundfos-express-tools\bronze impeller removal\output files\Revised Lbom-ES.xlsx'
# tabName = sheetname + " No Bronze"

# with pd.ExcelWriter(outputPath, engine='openpyxl', mode='a', if_sheet_exists='overlay') as writer:  
#             keep.to_excel(writer, sheet_name=tabName, index=False, startrow=psd_startrow-1, startcol=psd_startcol)
#             removals.to_excel(writer, sheet_name=tabName, index=False, header=False, startrow=append_location+1, startcol=psd_startcol)

In [ ]:
# df=pd.read_excel(outputPath,'Impeller',header=None)
# header=df.loc[:4,:]
# rest=df.loc[5:,:]
# rest.reset_index(drop=True, inplace=True)
# rest.columns=rest.loc[0,:]
# rest=rest.drop(0)
# from typing import overload,Literal,Sequence,Hashable,Iterable,Callable
# from pandas._typing import DtypeArg,StorageOptions
# from pandas.util._decorators import Appender


outputPath=r'C:\Projects\2022\Michaels_Code\grundfos-express-tools\bronze impeller removal\output files\Revised Lbom-ES.xlsx'




# def _find_header_end(df:DataFrame)->int:
#     for row in df.itertuples():
#         if row[1]=='[START]':
#             return int(row[0])


# @overload
# def _get_df(
#     io,
#     # sheet name is str or int -> DataFrame
#     sheet_name: str | int,
#     header: int | Sequence[int] | None = ...,
#     names=...,
#     index_col: int | Sequence[int] | None = ...,
#     usecols=...,
#     squeeze: bool | None = ...,
#     dtype: DtypeArg | None = ...,
#     engine: Literal["xlrd", "openpyxl", "odf", "pyxlsb"] | None = ...,
#     converters=...,
#     true_values: Iterable[Hashable] | None = ...,
#     false_values: Iterable[Hashable] | None = ...,
#     skiprows: Sequence[int] | int | Callable[[int], object] | None = ...,
#     nrows: int | None = ...,
#     na_values=...,
#     keep_default_na: bool = ...,
#     na_filter: bool = ...,
#     verbose: bool = ...,
#     parse_dates=...,
#     date_parser=...,
#     thousands: str | None = ...,
#     decimal: str = ...,
#     comment: str | None = ...,
#     skipfooter: int = ...,
#     convert_float: bool | None = ...,
#     mangle_dupe_cols: bool = ...,
#     storage_options: StorageOptions = ...,
# ) -> tuple[DataFrame,DataFrame,int]:
#     ...


# @Appender(pd.read_excel.__doc__,join='\n')
# def _get_df(**kwargs):
#     """This extends the pandas read_excel method for our uses.
#     Params:
#     **kwargs: Takes in keyword arguments only. These arguments also inherit from the 
#     """
#     df=pd.read_excel(**kwargs) #Reading the dataframe
#     header_size=_find_header_end(df) #Getting the size of the header

#     #Need to get the other dataframes
#     psd_df=df.loc[header_size-1:,:]
#     psd_df.reset_index(drop=True,inplace=True)
#     psd_df.columns=psd_df.loc[0,:]
#     psd_df=psd_df.drop(0)
#     original_length=len(df)

#     #Lets Check if we are using the header df
#     if bool(kwargs.get('use_header',False)):
#         if kwargs['use_header']==True:
#             header=df.loc[:header_size-2,:]
#             return header,psd_df,original_length
#         else:
#             return psd_df,original_length
#     return psd_df,original_length


        
        

    
    
    
    
    


Process Sheet

In [1]:
import datetime
import pandas as pd
import os
import numpy as np
from _types._types import FilePath
from pandas._typing import FilePath, ReadBuffer
from typing import Union,ParamSpec,Callable
from utils.file_ops import read_files_in_dir
from utils.Dataframe_tools import PSD_BOM_Updates
from pandas import DataFrame
from openpyxl import load_workbook,worksheet,Workbook
from openpyxl.styles.borders import Border,Side
from openpyxl.styles import Alignment,PatternFill

MYDIR= r"C:\Projects\2022\Michaels_Code\grundfos-express-tools\bronze impeller removal\input files"


def add_removal_note():
    timeStamp = datetime.datetime.now().strftime("%m-%d-%Y")
    reason = input("Reason for removal: ")
    authority = input("Who authorized/requested this change? ")
    return timeStamp + " " + reason + " per " + authority


def group_then_separate_by(psd_data:DataFrame,list_of_cols: list, pn_col: str, str_list:list[str])->tuple[DataFrame,DataFrame]:
    """list of cols are grouping categories. pn_col is the column that contains pns to find matches on."""
    groups = psd_data.groupby(list_of_cols) # Had to play around with this to get the right groupings

    df_list_to_remove = [] # will hold list of dataframes to be removed. Will concatenate to 1 dataframe later
    df_list_to_keep   = [] # will hold list of dataframes to remain in PSD. Will concatenate to 1 dataframe later

    # Iterates through each sub-group, checking if there is a pn that meets criteria for removal
    for _, frame in groups:
        if any(frame[pn_col].isin(str_list)):   # Finding matching partnumbers to remove
            df_list_to_remove.append(frame)     # Need to add this sub-group to a "removed dataframe"
        else:
            df_list_to_keep.append(frame)       # Need to retain this sub-group, add to a "keep dataframe"

    # Concatenating list of dfs to single dfs.
    removals = pd.concat(df_list_to_remove)
    keep = pd.concat(df_list_to_keep)
        
    return removals, keep

#list of sheet lists can not be empty 

list_sheet_lists=[['Impeller']]
target_dir=r'C:\Projects\2022\Michaels_Code\grundfos-express-tools\bronze impeller removal\output files'

list_of_pns = [
    96699290, 97775274, 96699299, 97775277, 96778078,
    97780992, 96699305, 96769184, 97778033, 96769190,
    96769205, 97778039, 96769256, 96896891, 96769259,
    96769271, 97780970, 96769280, 97780973
]

str_list = [str(numstr) for numstr in list_of_pns]


def process_sheet(file: tuple[Union[FilePath, ReadBuffer[bytes], bytes],str],
sheet_name: str | list[str],
removal_note:str|None|bool=None,
use_header:bool=False
)->tuple[str,
Union[tuple[DataFrame, int], tuple[DataFrame, DataFrame, int]],
str,
str]:
    """
    Params:
    data: Data comes from the binary of the file or the file path. 
    sheet_name: The name of the sheet to process.
    removal_note: Does the sheet have its own removal note or does it share one with the entire workbook (file). The default is set to it shares a note."""
    #Check if sheetname is a valid sheet in the file
    data,fname=file
    try:
        if not sheet_name in pd.ExcelFile(data).sheet_names:
            raise ValueError(f"Worksheet named '{sheet_name}' not found")
    except ValueError as err:
        print(err)
        return
    #Need to get the header dataframe and the psd_dataframe
    header,psd_data,length=PSD_BOM_Updates()._get_df(io=data,sheet_name=sheet_name,use_header=use_header,header=None)
    #length = tuple of (psd_start_row,original size)
    if not bool(removal_note):
        removal_note=add_removal_note()
    if header:
        return fname,header,psd_data,length,removal_note,sheet_name
    return fname,psd_data,length,removal_note,sheet_name

    

def process_file(file:FilePath,sheet_list:list[str],output_dir:str,one_removal_note:bool=False):
    #Does this file use the same removal note for every sheet
    if one_removal_note:
        removal_note=add_removal_note()
    else:
        removal_note=None
    if not isinstance(sheet_list,list):
        raise Exception(f'You must give the sheets as a list to {process_file.__name__}')
    output_file_name=os.path.join(output_dir,"Revised "+os.path.basename(file[1]))
    ret=[]
    for sheet in sheet_list:
        val=list(process_sheet(file,sheet,removal_note))
        val.extend([output_file_name])
        ret.append(val)
    return ret

        

def process_dir(dir,list_sheet_lists,output_dir:str,one_removal_note:bool):
    files=read_files_in_dir(dir)
    ret=[]
    for i,file in enumerate(files):
        if file[1].endswith(('.xls','.xlsx','.xlsm','.xlsb','.odf','.odt')):
            ret.extend(process_file(file,list_sheet_lists[i],output_dir,one_removal_note))
    return ret




grouping_func=(group_then_separate_by,[["Model", "Price ID"] ,"BOM"],{'str_list':list_of_pns})

tasks=process_dir(MYDIR,[['Impeller'],['Impeller'],['Impeller'],['Impeller'],['Impeller'],['Impeller']],target_dir,True)

def add_function(tasks,function,*args,**kwargs):
    for task in tasks:
        task.insert(2,(function,list(args),dict(kwargs)))
    return tasks

add_function(tasks,group_then_separate_by,["Model", "Price ID"] ,"BOM",str_list=list_of_pns)


def write_new_PSD(file_name,
psd_data,
grouping_func:Callable[[ParamSpec.args],DataFrame],
length:tuple[int,int],
removal_note,
sheet_name,
outPut_file_path
):
    print("Opening file for updates: {}".format(os.path.basename(file_name)))
    psd_startrow=length[0]
    psd_startcol=0
    end_row = psd_data[psd_data['Full Data'] == '[END]'].index.to_list()[0]
    psd_data = psd_data.iloc[:end_row]
    removals, keep = grouping_func[0](psd_data,*grouping_func[1],**grouping_func[2])
    removals.loc[removals["Full Data"] == "[START]", "Full Data"] = ""
    new_row = pd.DataFrame({'ID': removal_note}, index =[0])
    removals = pd.concat([new_row, removals[:]]).reset_index()
    column_list = keep.columns
    removals = removals[column_list]
    removals.sort_values(by=['ID'], inplace=True)
    keep.loc[0,'Full Data'] = np.nan
    keep.sort_values(by=['ID'], inplace=True)
    keep.reset_index(drop=True, inplace=True)
    keep.loc[0,'Full Data'] = "[START]"
    num_rows = len(keep)                      
    keep.loc[num_rows,'Full Data']="[END]"
    append_location = length[1] - len(removals)
    after_end_row = num_rows + psd_startrow + 2
    wb = load_workbook(file_name)
    ws = wb[sheet_name]
    tabName = sheet_name + "Modified"
    wb.copy_worksheet(ws).title = tabName
    ws_modified = wb[tabName]
    ws_modified.delete_rows(after_end_row, len(removals)-1)
    end_cell=ws.cell(row=after_end_row-1,column=1)
    end_cell.fill=PatternFill(fill_type='solid',start_color="FFCC99",end_color="FFCC99")
    for rows in ws_modified.iter_rows(min_row=append_location+2, max_row=append_location+2, min_col=1, max_col=40):
        for cell in rows:
            cell.fill = PatternFill(start_color="FF0000", end_color="FF0000", fill_type="solid")
    col=ws_modified.column_dimensions['A']
    col.border=Border(right=Side(style='thick'))
    wb.save(outPut_file_path)
    del(ws,ws_modified,wb,end_cell,col)
    with pd.ExcelWriter(outPut_file_path, engine='openpyxl', mode='a', if_sheet_exists='overlay') as writer:  
            keep.to_excel(writer, sheet_name=tabName, index=False, startrow=psd_startrow-1, startcol=psd_startcol)
            removals.to_excel(writer, sheet_name=tabName, index=False, header=False, startrow=append_location+1, startcol=psd_startcol)
    print("Closing file: {}".format(os.path.basename(file_name)),"\n",f"Wrote file: {os.path.basename(outPut_file_path)} to {os.path.dirname(outPut_file_path)}")

for task in tasks:
    write_new_PSD(*task)

# write_new_PSD(*a[0])

Opening file for updates: Lbom-ES.xlsx
Closing file: Lbom-ES.xlsx 
 Wrote file: Revised Lbom-ES.xlsx to C:\Projects\2022\Michaels_Code\grundfos-express-tools\bronze impeller removal\output files
Opening file for updates: LCS-bom-ES.xlsx
Closing file: LCS-bom-ES.xlsx 
 Wrote file: Revised LCS-bom-ES.xlsx to C:\Projects\2022\Michaels_Code\grundfos-express-tools\bronze impeller removal\output files
Opening file for updates: LFE_bom-ES.xlsx
Closing file: LFE_bom-ES.xlsx 
 Wrote file: Revised LFE_bom-ES.xlsx to C:\Projects\2022\Michaels_Code\grundfos-express-tools\bronze impeller removal\output files
Opening file for updates: VL-VLSbom.xlsx


ValueError: Grouper for 'Model' not 1-dimensional

In [2]:
psd_data_1=tasks[0][1]

psd_data_1.iloc[:]


,Full Data,QP,ID,a,No Name 1,Model,CodeX,OptionID,Material,PACOMatlCode,...,BOM,Description,Price ID,LeadtimeID,Days,No Name 2,No Name 3,No Name 4,No Name 5,No Name 6
0,[START],N,Price_BOM_L_Imp_5,5,NaN,:10707-LC:10707-LCV:,X0,ImpMatl_SS_AISI-304,"Stainless Steel, AISI-304",H304,...,98876008,NaN,A102324,LT027,0,NaN,NaN,NaN,NaN,NaN
1,NaN,N,Price_BOM_L_Imp_11,11,NaN,:10707-LC:10707-LCV:10707-LF:,X3,ImpMatl_SS_AISI-304,"Stainless Steel, AISI-304",H304,...,98876012,NaN,A102326,LT027,0,NaN,NaN,NaN,NaN,NaN
2,NaN,N,Price_BOM_L_Imp_18,18,NaN,:12501-LC:12501-LCV:,X0,ImpMatl_SS_AISI-304,"Stainless Steel, AISI-304",H304,...,RTF,NaN,A102328,LT027,0,NaN,NaN,NaN,NaN,NaN
3,NaN,N,Price_BOM_L_Imp_25,25,NaN,:12507-LC:12507-LCV:,X0,ImpMatl_SS_AISI-304,"Stainless Steel, AISI-304",H304,...,RTF,NaN,A102330,LT027,0,NaN,NaN,NaN,NaN,NaN
4,NaN,N,Price_BOM_L_Imp_36,36,NaN,:15509-LC:15509-LCV:,X0,ImpMatl_SS_AISI-304,"Stainless Steel, AISI-304",H304,...,RTF,NaN,A102333,LT027,0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
748,NaN,N,Price_BOM_L_Imp_1452,1452,NaN,:30507-LC:30507-LCV:,X3,ImpMatl_NiAl-Bronze_ASTM-B148_C95400,Nickel Aluminum Bronze ASTM B148 UNS C95400,B22,...,RTF,NaN,A102236,LT250,NaN,NaN,NaN,NaN,NaN,NaN
749,NaN,N,Price_BOM_L_Imp_1561,1561,NaN,:30507-LC:30507-LCV:,X3,ImpMatl_SS_AISI-304,"Stainless Steel, AISI-304",H304,...,99837749,NaN,A102375,LT250,126,NaN,NaN,NaN,NaN,NaN
750,NaN,N,Price_BOM_L_Imp_1746,1746,NaN,:30507-LC:30507-LCV:,X3,ImpMatl_NiAl-Bronze_ASTM-B148_C95400,Nickel Aluminum Bronze ASTM B148 UNS C95400,B22,...,RTF,NaN,A102236,LT250,NaN,NaN,NaN,NaN,NaN,NaN
751,NaN,N,Price_BOM_L_Imp_1855,1855,NaN,:30507-LC:30507-LCV:,X3,ImpMatl_SS_AISI-304,"Stainless Steel, AISI-304",H304,...,RTF,NaN,A101852,LT250,126,NaN,NaN,NaN,NaN,NaN


In [ ]:
import numpy as np


def name_blank_cols(df):
    col=df.columns.to_list()
    count=1
    for i,item in enumerate(col.copy()):
        if str(item)=='nan':
            col[i]=f"No Name {count}"
            count+=1
    df.columns=col
    

name_blank_cols(psd_data_1)
psd_data_1

In [7]:
from utils.file_ops import get_files_in_dir
sheet_list=[]
sheet='Impeller'
for file in get_files_in_dir(MYDIR,('.xlsx')):
    sheet_list.append([sheet])

Making a New File

In [ ]:

from _utils.excel_tools import write_excel_file
from _utils.file_ops import add_str_to_filename
from _utils.Dataframe_tools import write_df_to_excel

header=process_sheet(file,'Impeller','No')
tabName = sheetname + " No Bronze"


with pd.ExcelWriter(outputPath, engine='openpyxl', mode='a', if_sheet_exists='overlay') as writer:
        keep.to_excel(writer, sheet_name=tabName, index=False,startrow=psd_startrow-1,startcol=psd_startcol)
        removals.to_excel(writer, sheet_name=tabName, index=False,header=False, startrow=append_location+1, startcol=psd_startcol)
        header.to_excel(writer, sheet_name=tabName, index=False,header=False, startrow=0, startcol=0)



Add Formatting

In [ ]:
from openpyxl.styles.borders import Border,Side
from openpyxl.styles.fonts import Font
from openpyxl.styles import Alignment,PatternFill
from openpyxl import load_workbook,worksheet,Workbook
from openpyxl.worksheet import cell_range


outputPath=r'C:\Projects\2022\Michaels_Code\grundfos-express-tools\bronze impeller removal\output files\Revised Lbom-ES.xlsx'
sheetname='Impeller'
tabName = sheetname + " No Bronze"
psd_startrow=get_start_row(wb,'Impeller',min_row=1,max_col=1)


wb=load_workbook(outputPath)
ws=wb[tabName]

#Adding the Border
col=ws.column_dimensions['A']
col.border=Border(right=Side(style='thick'))

#Fitting the Cell Widths in the Columns
dims = {}
for row in ws.iter_rows(min_row=0):
    for cell in row:
        if cell.value:
            dims[cell.column_letter] = max((dims.get(cell.column_letter, 0)+2, len(str(cell.value))+2))
            if row[0].row <= psd_startrow and cell.column_letter=='A':
                cell.border=Border(right=Side(style='thick'))
            elif cell.row<psd_startrow and cell.value=='[END]': #Gets the column end
                cell.fill=PatternFill(fill_type='solid',start_color="FFCC99",end_color="FFCC99")

                #Applying Top Rows Formatting
                for row2 in ws.iter_rows(min_col=0,max_col=cell.column-1,min_row=0,max_row=psd_startrow-1):
                    for cell2 in row2:
                        if cell2.row==1:
                            cell2.fill=PatternFill(fill_type='solid',start_color="808080",end_color="808080")
                            cell2.font=Font(color='00FFFFFF')
                            if cell2.column==1:
                                cell2.font=Font(bold=True,color='00FFFFFF')
                                cell2.border=Border(right=Side(style='thick'),bottom=Side(style='thick'))
                            else:
                                cell2.border=Border(bottom=Side(style='thick'))
                        elif cell2.row==cell.row-1:
                            cell2.fill=PatternFill(fill_type='solid',start_color="FFCC99",end_color="FFCC99")
                            if cell2.column==1:
                                cell2.border=Border(right=Side(style='thick'),bottom=Side(style='thin'))
                            else:
                                cell2.border=Border(bottom=Side(style='thin'))
                        elif cell2.row==psd_startrow-1:
                            cell2.fill=PatternFill(fill_type='solid',start_color="FFCC99",end_color="FFCC99")
                            if cell2.column==1:
                                cell2.border=Border(right=Side(style='thick'),bottom=Side(style='thick'))
                            else:
                                cell2.border=Border(bottom=Side(style='thick'))
                        else:
                            cell2.fill=PatternFill(fill_type='solid',start_color="FFCC99",end_color="FFCC99")
                        if cell2.column>1:
                            cell2.alignment=Alignment(horizontal='center')
                        else:
                            cell2.alignment=Alignment(horizontal='left')
                flt=str(cell_range.CellRange(min_col=2, min_row=psd_startrow, max_col=cell.column-1, max_row=psd_startrow))
                ws.auto_filter.ref=flt     
            elif cell.row>psd_startrow and cell.column==1 and cell.value=='[START]':
                cell.fill=PatternFill(fill_type='solid',start_color="FFCC99",end_color="FFCC99")
                cell.alignment=Alignment(horizontal='right')
            
            elif cell.value=='[END]' and cell.row>psd_startrow and cell.column==1:
                cell.fill=PatternFill(fill_type='solid',start_color="FFCC99",end_color="FFCC99")
            
            elif cell.row==append_location+2 and cell.column>=1 and cell.column<=40:
                cell.fill = PatternFill(start_color="FF0000", end_color="FF0000", fill_type="solid")
                                   
for col, value in dims.items():
    ws.column_dimensions[col].width = value


wb.save(outputPath)
    


- it is faster to just use the copy code

In [ ]:
import glob
path=r'C:\Projects\2022\Michaels_Code\grundfos-express-tools\bronze impeller removal\input files'
types=[]
pd.read_excel
glob.glob(path+'\*.*')